In [3]:
import os
import logging

import psycopg2
import psycopg2.extensions
from pymongo import MongoClient
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, Float, MetaData, String
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

In [4]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

# Задание по Psycopg2
# --------------------------------------------------------------

In [6]:
params = {
    "host": 'localhost', #os.environ['APP_POSTGRES_HOST']
    "port": '5433', #os.environ['APP_POSTGRES_PORT']
    "user": 'postgres'
}
conn = psycopg2.connect(**params)

In [8]:
# дополнительные настройки
psycopg2.extensions.register_type(
    psycopg2.extensions.UNICODE,
    conn
)
conn.set_isolation_level(
    psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT
)
cursor = conn.cursor()
conn = psycopg2.connect(**params)

# дополнительные настройки
psycopg2.extensions.register_type(
    psycopg2.extensions.UNICODE,
    conn
)
conn.set_isolation_level(
    psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT
)
cursor = conn.cursor()

In [10]:
# ВАШ КОД ЗДЕСЬ
# -------------
# таблица movies_top
# movieId (id фильма), ratings_num(число рейтингов), ratings_avg (средний рейтинг фильма)

Psycopg2
Необходимо с помощью Psycorg выполнить создание таблицы movies_top c помощью конструкции

    SELECT *  INTO movies_top FROM (ЗАПРОС);
Где ЗАПРОС имеет следующий вид:

три поля: movieId (id фильма), ratings_num(число рейтингов), ratings_avg (средний рейтинг фильма)
В выборку должны попасть только фильмы, у которых средний рейтинг выше 3.


In [19]:
user_item_query_config = {
    "MIN_AVG_RATING": 3
}

In [79]:
sql_str_1 = (
        """
          SELECT * INTO movies_top FROM 
          (
          SELECT 
          movieId, 
          count(rating) as ratings_num, 
          avg(rating) as ratings_avg 
          FROM ratings 
          GROUP BY movieId 
          HAVING AVG(rating)>%(MIN_AVG_RATING)d) as tmp
        """ % user_item_query_config
)

cursor.execute(sql_str_1)

In [80]:
sql_str_2 = (
        """
          SELECT * FROM movies_top
          
        """
)

 # вложенный запрос
 SELECT
      movieId,
      count(rating) as ratings_num,
      avg(rating) as ratings_avg
  FROM ratings
  GROUP BY movieId
  HAVING AVG(rating)>3

In [81]:
# выгружаем данные из БД в Python
cursor.execute(sql_str_2)
ui_data = [a for a in cursor.fetchall()]
conn.commit()
logger.info("Данные по оценкам загружены из Postgres")

2018-12-27 19:58:47,213 : INFO : Данные по оценкам загружены из Postgres


In [82]:
agg_filename = '/Users/olgamonahova/netology_data/HW1_v2.tsv'
# создаём текстовый файл с результатами
with open(agg_filename, 'w') as f:
    for row in ui_data:
        f.write("{}\t{}\t{}\n".format(row[0], row[1], row[2]))
logger.info("Данные сохранены в {}".format(agg_filename))

2018-12-27 19:58:53,227 : INFO : Данные сохранены в /Users/olgamonahova/netology_data/HW1_v2.tsv


# Задание по SQLAlchemy
# --------------------------------------------------------------
Base = declarative_base()


class MoviesTop(Base):
    __tablename__ = 'movies_top'

    movieid = Column(Integer, primary_key=True)
    ratings_num = Column(Float)
    ratings_avg = Column(Float)

    def __repr__(self):
        return "<User(movieid='%s', ratings_num='%s', ratings_avg='%s')>" % (self.movieid, self.ratings_num, self.ratings_avg)


# Создаём сессию

engine = create_engine('postgresql://postgres:@{}:{}'.format(os.environ['APP_POSTGRES_HOST'], os.environ['APP_POSTGRES_PORT']))
Session = sessionmaker(bind=engine)
session = Session()


# --------------------------------------------------------------
# Ваш код здесь
# выберите контент у которого больше 15 оценок (используйте filter)
# и средний рейтинг больше 3.5 (filter ещё раз)
# отсортированный по среднему рейтингу (используйте order_by())
# id такого контента нужно сохранить в массив top_rated_content_ids

top_rated_query = session.query(MoviesTop)

logger.info("Выборка из top_rated_query\n{}".format([i for i in top_rated_query.limit(4)]))

top_rated_content_ids = [
    i[0] for i in top_rated_query.values(MoviesTop.movieid)
][:5]
# --------------------------------------------------------------

In [84]:
import os

from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, Float, MetaData
from sqlalchemy.orm import mapper
from sqlalchemy.orm import sessionmaker

Base = declarative_base()

In [85]:
class MoviesTop(Base):
    __tablename__ = 'movies_top'

    movieid = Column(Integer, primary_key=True)
    ratings_num = Column(Float)
    ratings_avg = Column(Float)

    def __repr__(self):
        return "<User(movieid='%s', ratings_num='%s', ratings_avg='%s')>" % (self.movieid, self.ratings_num, self.ratings_avg)

In [86]:
engine = create_engine('postgresql://postgres:@{}:{}'.format('localhost', '5433'))
Session = sessionmaker(bind=engine)
session = Session()

In [87]:
top_rated_query = session.query(MoviesTop)

logger.info("Выборка из top_rated_query\n{}".format([i for i in top_rated_query.limit(4)]))

top_rated_content_ids = [
    i[0] for i in top_rated_query.values(MoviesTop.movieid)
][:5]

2018-12-27 21:41:28,776 : INFO : Выборка из top_rated_query
[<User(movieid='790', ratings_num='6.0', ratings_avg='3.16666666666667')>, <User(movieid='146688', ratings_num='2.0', ratings_avg='3.25')>, <User(movieid='69906', ratings_num='1.0', ratings_avg='5.0')>, <User(movieid='26745', ratings_num='5.0', ratings_avg='3.3')>]
